# PLIP — Protein ligand interaction profiling

This notebook will walk us through using PLIP for identification of non-covalent interactions between biological macromolecules and their ligands.

**PLIP implementation paper**:
Adasme et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. NAR 2021

# 0) Complete example
See the [sample notebook](/Quickstarts/protein-ligand-interactions-sample.ipynb) for a complete demonstration.

# 1) Setup
See [Quickstart](../index.ipynb#imports) for more details on the setup.


## 1.0) Imports

In [ ]:
import os
from pathlib import Path

import rush
from pdbtools import pdb_fetch

## 1.1) Configuration

In [ ]:
TOKEN = os.getenv("RUSH_TOKEN")
# You might have a custom deployment url, by default the client will use
# https://tengu.qdx.ai
RUSH_URL = os.getenv("RUSH_URL")

Let's set some global variables that define our project.

In [ ]:
DESCRIPTION = "plip-inference-notebook"
TAGS = ["rush-py", "plip", "notebook"]

## 1.2) Build your client

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / "plip-inference-rush-py-demo"
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR, access_token=TOKEN, url=RUSH_URL)
    await client.nuke(remote=False)
os.makedirs(WORK_DIR, exist_ok=True)

In [ ]:
client = await rush.build_provider_with_functions(
    access_token=TOKEN, url=RUSH_URL, batch_tags=TAGS
)

In [ ]:
# |hide
client = await rush.build_provider_with_functions(
    access_token=TOKEN,
    url=RUSH_URL,
    workspace=WORK_DIR,
    batch_tags=TAGS,
    restore_by_default=True,
)

# 2) Preparation

## 2.0) Fetch example PDB
Note that PLIP requires a PDB file of a protein-ligand complex.

In [ ]:
PDB_ID = "1GIH"
FILE_NAME = f"{PDB_ID}.pdb"
FILE_PATH = WORK_DIR / FILE_NAME

In [ ]:
complex = list(pdb_fetch.fetch_structure(PDB_ID))

with open(FILE_PATH, "w") as f:
    for line in complex:
        f.write(line)

# 3) PLIP

## 3.1) Arguments

In [ ]:
help(client.plip)

Help on function plip in module rush.provider:

async plip(*args: *tuple[RushObject[None]], target: 'Target | None' = None, resources: 'Resources | None' = {'storage': 138, 'storage_units': 'MB', 'gpus': 0}, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[Record], RushObject[None]]
    Run PLIP on the input PDB file of a protein-ligand complex to obtain interaction data.


    Module version:
    `github:talo/tengu-plip/b0119d6e16e564376dee67f5deb418f03b818065#plip_tengu`

    QDX Type Description:

        input_complex_pdb_file: Object[@$PDB]
        ->
        output_conformer: Object[Conformer];
        output_txt_file: Object[@$PDB]


    :param input_complex_pdb_file: The protein-ligand complex as a PDB file; ligand should be the only HETATM lines.
    :return output_conformer: A QDX Conformer of the complex with the computed interaction data
    :return output_txt_file: The output txt report from the PLIP run



We can see from the above help documentation that we need to pass the Path to our PDB file as an argument.

## 3.2) Run PLIP
Finally, we run PLIP so we can identify interactions between the ligand and the protein within our protein complex.

In [ ]:
PLIP_RESOURCES = {"storage": 1024_000}

output_conformer, output_txt = await client.plip(
    FILE_PATH, resources=PLIP_RESOURCES
)

2024-02-28 18:51:38,266 - rush - INFO - Trying to restore job with tags: ['rush-py', 'plip', 'notebook'] and path: github:talo/tengu-plip/b0119d6e16e564376dee67f5deb418f03b818065#plip_tengu


## 3.3) Get output conformer
Here, we get the output Conformer (in QDXF format) that is enriched with `binding_site_interactions` by PLIP.

In [ ]:
output_conformer = await output_conformer.get()

2024-02-28 18:51:38,623 - rush - INFO - Argument f8ff9ef9-28d4-49db-998d-2e56004805c1 is now ModuleInstanceStatus.RESOLVING
2024-02-28 18:51:42,021 - rush - INFO - Argument f8ff9ef9-28d4-49db-998d-2e56004805c1 is now ModuleInstanceStatus.ADMITTED
2024-02-28 18:51:55,231 - rush - INFO - Argument f8ff9ef9-28d4-49db-998d-2e56004805c1 is now ModuleInstanceStatus.DISPATCHED
2024-02-28 18:52:00,789 - rush - INFO - Argument f8ff9ef9-28d4-49db-998d-2e56004805c1 is now ModuleInstanceStatus.AWAITING_UPLOAD


## 3.4) Get PLIP output document
PLIP also returns a link to the output document of the results of the PLIP output.

In [ ]:
await output_txt.download("plip_output.txt", overwrite=True)
with open(client.workspace / "objects" / "plip_output.txt") as f:
    print(f.readline(), "...")

Prediction of noncovalent interactions for PDB structure 1GIH
 ...
